In [1]:
PATH = '/home/g056122/FeatExt_Data_Clustering/final_github/content_based_filtering'

In [2]:
DATAPATH = '/home/g056122/ISG_studienarbeit/Movielens_100K'

In [3]:
''' 
load 'validate' and 'valres' module from 'Module' folder
'''
# import required libraries
import numpy as np
import pandas as pd
import datetime
from sklearn.cluster import OPTICS
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt
from validate import evaluation
from valres import result

In [4]:
# import movies data
dfmovies = pd.read_csv("/home/g056122/ISG_studienarbeit/Movielens_100K/u.item", sep='|', header=None, engine='python', encoding='latin-1')
dfmovies.columns = ['movieId', 'movietitle', 'releasedate', 'videoreleasedate','imdburl', 'unknown', 'action', 'adventure', 'animation','childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film-Noir', 'horror', 'musical', 'mystery', 'romance', 'sci-fi','thriller', 'war', 'western']
copydfmovies = dfmovies.copy()

In [5]:
dfmovies = dfmovies.drop(['videoreleasedate', 'imdburl'], axis=1)

In [6]:
# feature transformation
dfmovies['releasedate'] = pd.to_datetime (dfmovies['releasedate'], format='%d-%b-%Y')
dfmovies['year']=dfmovies['releasedate'].dt.year

In [7]:
dfmovies.head()

,movieId,movietitle,releasedate,unknown,action,adventure,animation,childrens,comedy,crime,...,film-Noir,horror,musical,mystery,romance,sci-fi,thriller,war,western,year
0,1,Toy Story (1995),1995-01-01,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1995.0
1,2,GoldenEye (1995),1995-01-01,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995.0
2,3,Four Rooms (1995),1995-01-01,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995.0
3,4,Get Shorty (1995),1995-01-01,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1995.0
4,5,Copycat (1995),1995-01-01,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1995.0


In [8]:
# drop features
dfmovies = dfmovies.drop(['movieId', 'movietitle', 'releasedate'], axis=1)

In [9]:
# encode year feature in dataset
n_dfmovies = pd.get_dummies(dfmovies, columns = ['year'])

In [10]:
n_dfmovies.head()

,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,...,year_1989.0,year_1990.0,year_1991.0,year_1992.0,year_1993.0,year_1994.0,year_1995.0,year_1996.0,year_1997.0,year_1998.0
0,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [11]:
n_dfmovies.columns

Index(['unknown', 'action', 'adventure', 'animation', 'childrens', 'comedy',
       'crime', 'documentary', 'drama', 'fantasy', 'film-Noir', 'horror',
       'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war', 'western',
       'year_1922.0', 'year_1926.0', 'year_1930.0', 'year_1931.0',
       'year_1932.0', 'year_1933.0', 'year_1934.0', 'year_1935.0',
       'year_1936.0', 'year_1937.0', 'year_1938.0', 'year_1939.0',
       'year_1940.0', 'year_1941.0', 'year_1942.0', 'year_1943.0',
       'year_1944.0', 'year_1945.0', 'year_1946.0', 'year_1947.0',
       'year_1948.0', 'year_1949.0', 'year_1950.0', 'year_1951.0',
       'year_1952.0', 'year_1953.0', 'year_1954.0', 'year_1955.0',
       'year_1956.0', 'year_1957.0', 'year_1958.0', 'year_1959.0',
       'year_1960.0', 'year_1961.0', 'year_1962.0', 'year_1963.0',
       'year_1964.0', 'year_1965.0', 'year_1966.0', 'year_1967.0',
       'year_1968.0', 'year_1969.0', 'year_1970.0', 'year_1971.0',
       'year_1972.0', 'year_1973

In [12]:
n_dfmovies.describe()

,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,...,year_1989.0,year_1990.0,year_1991.0,year_1992.0,year_1993.0,year_1994.0,year_1995.0,year_1996.0,year_1997.0,year_1998.0
count,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.00000,...,1682.000000,1682.000000,1682.00000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000
mean,0.001189,0.149227,0.080262,0.024970,0.072533,0.300238,0.064804,0.029727,0.431034,0.01308,...,0.008918,0.014269,0.01308,0.021998,0.074911,0.127229,0.130202,0.211058,0.170036,0.038644
std,0.034473,0.356418,0.271779,0.156081,0.259445,0.458498,0.246253,0.169882,0.495368,0.11365,...,0.094041,0.118632,0.11365,0.146719,0.263326,0.333329,0.336626,0.408181,0.375776,0.192803
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
# OPTICS model
optics_model = OPTICS()
optics_model.fit(n_dfmovies)

/home/g056122/miniconda3/envs/my_env/lib/python3.10/site-packages/sklearn/cluster/_optics.py:932: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


OPTICS()

In [14]:
# clusters 
cluster = np.unique(optics_model.labels_)
print('Clusters: ', cluster)

Clusters:  [-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46
 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61]


In [15]:
#outliers
outliers = np.count_nonzero(optics_model.labels_ == -1)
print('Outliers: ', outliers)

Outliers:  770


In [16]:
# determine cluster representatives
labels = optics_model.labels_
reach_dis = optics_model.reachability_
cluster_representatives = []
for i in np.unique(optics_model.labels_):
    cd = np.min(reach_dis[labels == i])
    cluster_representatives.append(cd)
# convert list to array
cluster_representatives = np.array(cluster_representatives)

In [17]:
# augment dataset
dfmovies['reach_dis'] = optics_model.reachability_
for j in range(len(dfmovies)):
    for m,n in enumerate(cluster_representatives):
        dfmovies[f'cluster {m}'] = dfmovies['reach_dis'] - n

In [18]:
dfmovies.head()

,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,...,cluster 53,cluster 54,cluster 55,cluster 56,cluster 57,cluster 58,cluster 59,cluster 60,cluster 61,cluster 62
0,0,0,0,1,1,1,0,0,0,0,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
1,0,1,1,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
2,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
3,0,1,0,0,0,1,0,0,1,0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
4,0,0,0,0,0,0,1,0,1,0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0


In [19]:
dfmovies = dfmovies.replace([np.inf, -np.inf], np.nan)
dfmovies = dfmovies.interpolate()

In [20]:
# fill empty values to dataset mean value
dfmovies = dfmovies.fillna(dfmovies.mean())

In [21]:
# merge movieId and movies dataset
dfmovies = pd.concat([copydfmovies['movieId'], dfmovies], axis=1)
dfmovies.head()

,movieId,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,...,cluster 53,cluster 54,cluster 55,cluster 56,cluster 57,cluster 58,cluster 59,cluster 60,cluster 61,cluster 62
0,1,0,0,0,1,1,1,0,0,0,...,0.634713,-0.365287,0.634713,-0.365287,0.634713,-0.365287,0.634713,0.634713,0.634713,0.634713
1,2,0,1,1,0,0,0,0,0,0,...,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
2,3,0,0,0,0,0,0,0,0,0,...,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
3,4,0,1,0,0,0,1,0,0,1,...,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
4,5,0,0,0,0,0,0,1,0,1,...,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
# import ratings dataset
dfratings = pd.read_csv(f"{DATAPATH}/u.data", sep='\t', header=None, engine='python', encoding='latin-1')
dfratings.columns = ['userId', 'movieId', 'rating', 'timestamp']

In [23]:
scaler = MinMaxScaler()
dfratings['timestamp'] = pd.to_datetime (dfratings['timestamp'], unit='s')
dfratings['year']=dfratings['timestamp'].dt.year
dfratings[['year', 'rating']]=scaler.fit_transform(dfratings[['year', 'rating']])
dfratings = dfratings.drop('timestamp', axis=1)

In [24]:
dfratings['year'].nunique()

2

In [25]:
# merge movies and ratings dataset
dfratings = pd.merge(dfmovies, dfratings, on='movieId')

In [26]:
dfratings.head()

,movieId,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,...,cluster 56,cluster 57,cluster 58,cluster 59,cluster 60,cluster 61,cluster 62,userId,rating,year_y
0,1,0,0,0,1,1,1,0,0,0,...,-0.365287,0.634713,-0.365287,0.634713,0.634713,0.634713,0.634713,308,0.75,1.0
1,1,0,0,0,1,1,1,0,0,0,...,-0.365287,0.634713,-0.365287,0.634713,0.634713,0.634713,0.634713,287,1.00,0.0
2,1,0,0,0,1,1,1,0,0,0,...,-0.365287,0.634713,-0.365287,0.634713,0.634713,0.634713,0.634713,148,0.75,0.0
3,1,0,0,0,1,1,1,0,0,0,...,-0.365287,0.634713,-0.365287,0.634713,0.634713,0.634713,0.634713,280,0.75,1.0
4,1,0,0,0,1,1,1,0,0,0,...,-0.365287,0.634713,-0.365287,0.634713,0.634713,0.634713,0.634713,66,0.50,0.0


In [27]:
# split dataset to X (features) and y (labels)
X = dfratings.drop('rating', axis=1).values
y = dfratings['rating'].values

In [28]:
# kfold cross validation
K = evaluation( X, y, 5, 100)
cv = K.kfold()

Fold 1:
Fold 2:
Fold 3:
Fold 4:
Fold 5:


In [29]:
#split tuple 
classes, eucdis = cv
classes, eucdis

(array([[3.54, 3.26, 3.58, ..., 2.72, 2.28, 2.11],
        [4.3 , 3.87, 3.84, ..., 2.81, 2.64, 3.1 ],
        [3.21, 3.86, 4.44, ..., 1.69, 2.91, 2.38],
        [4.28, 3.62, 4.8 , ..., 2.04, 1.76, 3.2 ],
        [4.66, 3.12, 3.72, ..., 2.88, 2.59, 2.14]]),
 array([[0.75, 0.75, 0.75, ..., 0.25, 0.  , 0.5 ],
        [0.75, 0.5 , 1.  , ..., 0.25, 0.5 , 0.5 ],
        [0.5 , 0.5 , 0.75, ..., 0.25, 0.75, 0.25],
        [1.  , 0.75, 0.5 , ..., 0.5 , 0.  , 0.5 ],
        [0.75, 0.5 , 0.75, ..., 0.5 , 0.75, 0.5 ]]))

In [30]:
# metrics
ans = result(classes, eucdis)
Metrics = ans.validate()

Metric | Mean | Standard Deviation
RMSE 2.965503347766258, 0.004339284170796251
MAE 2.8932217000000002, 0.004556224408432909
